- DataFrame由记录（record）组成，record是Row类型，一条record由多列组成
- 列是表达式，对Row的每一列进行计算
- 模式定义了DataFrame的列名和列的数据类型
- DataFrame的分区定义了DataFrame以及Dataset在集群上的物理分布
- 划分模式定义了partition的分配方式

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Python").getOrCreate()

In [3]:
df = spark.read.format("json").load("./data/flight-data/json/2015-summary.json")

In [3]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



<h4>模式</h4>

- 模式定义了DataFrame的列名以及数据类型

In [4]:
df.schema

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))

- 可以看到一个模式是由许多字段构成的StructType，这些字段为StructField
- StructType具有名称、类型和布尔标志（标志指示该列是否可以包含缺失值和空值）
- 用户可以指定该列关联的元数据（metadata），元数据存储着有关此列的信息
- 以下示例演示了如何为一个DataFrame创建并指定模式

In [5]:
from pyspark.sql.types import StructField, StructType, StringType, LongType

myManualSchema = StructType([
    StructField("DEST_COUNTRY_NAME", StringType(), True),
    StructField("ORIGIN_COUNTRY_NAME", StringType(), True),
    StructField("count", LongType(), False, metadata={"hello":"world"})
])

df = spark.read.format("json").schema(myManualSchema)\
.load("./data/flight-data/json/2015-summary.json")

<h4>列和表达式</h4>

- 所谓表达式，指的是我们对DataFrame的某一个记录的一个或多个值的一组转换操作，就像函数一样
- 我们将列名作为表达式的输入并解析，然后针对数据集中的每条记录应用表达式

In [6]:
from pyspark.sql.functions import col, column

In [7]:
col("someColumnName")
df['count']

Column<'count'>

<h4>列的转换操作</h4>

- 列既然是表达式，当然可以进行转换操作
- 最简单的就是对DataFrame列的引用，expr("someCol")等价于col("someCol")
- expr("someCol - 5")与执行col("someCol") - 5，都是相同的转换操作，都会被编译为表示操作顺序的逻辑树
- 表达式等价于有效SQL代码，SQL与DataFrame代码在执行前会被编译成相同的底层逻辑树
- SQL代码和DataFrame代码性能是一样的


- DataFrame的每一行都是一个记录，记录是Row类型的对象
- Spark使用列表达式操纵Row类型对象
- Row对象内部其实是字节数组，但是Spark没有提供访问这些数组的接口，只能用列表达式去操纵

In [8]:
from pyspark.sql.functions import expr
expr("(((someCol + 5) * 200) - 6) < otherCol")

Column<'((((someCol + 5) * 200) - 6) < otherCol)'>

In [9]:
df.columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

<b>当使用DataFrame时，向驱动器请求行的命令总是返回一个或多个Row类型的行数据</b>

In [10]:
df.first()

Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15)

<h4>创建行</h4>

- 可以手动实例化一个Row对象来创建行
- DataFrame有模式，但是行对象本身没有，创建时需要注意按照DataFrame的列顺序来初始化Row对象

In [11]:
from pyspark.sql import Row
myRow = Row("Hello", None, 1, False)
myRow[0]

'Hello'

<h4>创建DataFrame</h4>

- 以下不仅创建DataFrame实例，也将其注册为临时视图，以便可以用SQL访问
- 以下是以读取的方式创建DataFrame

In [39]:
df = spark.read.format("json").load("./data/flight-data/json/2015-summary.json")
df.createOrReplaceTempView("dfTable")

In [53]:
from pyspark.sql import Row
from pyspark.sql.types import StructField, StructType, StringType, LongType
myManualSchema = StructType([
    StructField("some", StringType(), True),
    StructField("col", StringType(), True),
    StructField("names", LongType(), False)
])
myRow = Row("Hello", None, 1)
myDf = spark.createDataFrame([myRow], myManualSchema)
myDf.show()

Py4JJavaError: An error occurred while calling o250.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 11.0 failed 1 times, most recent failure: Lost task 0.0 in stage 11.0 (TID 22) (26.26.26.1 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:182)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:145)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:373)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:337)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint

<h4>select函数和selectExpr函数</h4>

- 支持在DataFrame上执行类似数据表的SQL查询

In [54]:
df.select("DEST_COUNTRY_NAME").show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



等价于：SELECT DEST_COUNTRY_NAME FROM dfTable LIMIT 2

In [55]:
df.select("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME").show(2)

+-----------------+-------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----------------+-------------------+
|    United States|            Romania|
|    United States|            Croatia|
+-----------------+-------------------+
only showing top 2 rows



In [12]:
from pyspark.sql.functions import expr, col, column
df.select("DEST_COUNTRY_NAME",
          expr("DEST_COUNTRY_NAME"),
          col("DEST_COUNTRY_NAME"),).show(2)

+-----------------+-----------------+-----------------+
|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-----------------+-----------------+-----------------+
|    United States|    United States|    United States|
|    United States|    United States|    United States|
+-----------------+-----------------+-----------------+
only showing top 2 rows



- expr可以引用一列，也可以是对列进行操纵的字符串表达式
- 以下先更改列名，再通过alias方法把列名改回去

In [13]:
df.select(expr("DEST_COUNTRY_NAME as destination").alias("DEST_COUNTRY_NAME"))

DataFrame[DEST_COUNTRY_NAME: string]

- Spark有一个可以有效描述expr的接口：selectExpr
- 使用selectExpr就可以直接在双引号中使用表达式，无须加上expr

In [17]:
df.selectExpr("DEST_COUNTRY_NAME as newColumnName", "DEST_COUNTRY_NAME").show(2)

+-------------+-----------------+
|newColumnName|DEST_COUNTRY_NAME|
+-------------+-----------------+
|United States|    United States|
|United States|    United States|
+-------------+-----------------+
only showing top 2 rows



- 以下再给一个例子，在DataFrame中增加了一个新列withinCountry，该列描述了destination和origin是否相同

In [18]:
df.selectExpr(
    "*", # all origin columns
    "(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry"
).show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



-- in SQL<br>
SELECT \*, (DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry<br>
FROM dfTable<br>
LIMIT 2

- 还可以使用聚合操作

In [19]:
df.selectExpr("avg(count)", "count(distinct(DEST_COUNTRY_NAME))").show(2)

+-----------+---------------------------------+
| avg(count)|count(DISTINCT DEST_COUNTRY_NAME)|
+-----------+---------------------------------+
|1770.765625|                              132|
+-----------+---------------------------------+



-- in SQL<br>
SELECT avg(count), count(distinct(DEST_COUNTRY_NAME)) FROM dfTable LIMIT 2

<h4>转换操作成Spark类型(字面量)</h4>

- 我们可以通过字面量(literal)为Spark传递一个显式的值
- 简单来说，就是将给定编程语言的字面上的值转换为Spark可以理解的值

In [21]:
from pyspark.sql.functions import lit
df.select(expr("*"), lit(1).alias("One")).show(2)

+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|One|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15|  1|
|    United States|            Croatia|    1|  1|
+-----------------+-------------------+-----+---+
only showing top 2 rows



-- in SQL<br>
SELECT \*, 1 as One FROM dfTable LIMIT 2

<h4>添加列</h4>

- 使用withColumn可以为df增加新列
- withColumn函数有两个参数：列名和为给定行赋值的表达式
- 也可以用该函数对列重命名，但是会保留原来的列，说到底还是创建了个新列

In [22]:
df.withColumn("numberOne", lit(1)).show(2)

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|numberOne|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|        1|
|    United States|            Croatia|    1|        1|
+-----------------+-------------------+-----+---------+
only showing top 2 rows



In [23]:
df.withColumn("withinCountry", expr("ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME")).show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [24]:
df.withColumn("Destination", expr("DEST_COUNTRY_NAME")).columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count', 'Destination']

<h4>重命名列</h4>

- 传入两个参数：要被修改的列名，新的列名
- 会替换原列

In [25]:
df.withColumnRenamed("DEST_COUNTRY_NAME", "dest").columns

['dest', 'ORIGIN_COUNTRY_NAME', 'count']

<h4>保留字与关键字</h4>

- 在表达式中引用列的时候，通常是双引号中直接写列名，这时如果有空格或连字符等保留字，会导致表达式解析错误
- 这时候就需要对字符串使用反引号
- 以下示例首先我们创建一个带空格的列，然后对在引用列时使用反引号

In [26]:
dfWithLongColName = df.withColumn(
    "This Long Column-Name",
    expr("ORIGIN_COUNTRY_NAME")
)

In [27]:
dfWithLongColName.selectExpr(
    "`This Long Column-Name`",
    "`This Long Column-Name` as `new col`" 
).show(2)

dfWithLongColName.createOrReplaceTempView("dfTableLong")

+---------------------+-------+
|This Long Column-Name|new col|
+---------------------+-------+
|              Romania|Romania|
|              Croatia|Croatia|
+---------------------+-------+
only showing top 2 rows



-- in SQL<br>
SELECT \`This Long Column Name\`, \`This Long Column Name\` as \`new col\` FROM dfTableLong LIMIT 2

<h4>区分大小写</h4>

- Spark默认不区分大小写
- 可以通过如下方式配置

-- in SQL<br>
set spark.sql.caseSensitive true

<h4>删除列</h4>

In [28]:
df.drop("ORIGIN_COUNTRY_NAME").columns
dfWithLongColName.drop("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME")

DataFrame[count: bigint, This Long Column-Name: string]

<h4>更改列的类型</h4>

- 可以将某一列类型转换操作为另一种类型

In [29]:
df.withColumn("count2", col("count").cast("long"))

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint, count2: bigint]

-- in SQL<br>
SELECT \*, cast(count as long) AS count2 FROM dfTable

<b>过滤行</b>

 - 有两种方式过滤DataFrame的行，分别是where和filter

In [30]:
df.filter(col("count") < 2).show(2)
df.where("count < 2").show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



-- in SQL<br>
SELECT * FROM dfTable WHERE count < 2 LIMIT 2

In [35]:
df.where(col("count") < 2).where(col("ORIGIN_COUNTRY_NAME") != "Croatia").show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|          Singapore|    1|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



-- in SQL<br>
SELECT * FROM dfTable WHERE count < 2 AND ORIGIN_COUNTRY_NAME != "Croatia"

<h4>获得去重后的行</h4>

In [37]:
df.select("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME").distinct().count()
df.select("ORIGIN_COUNTRY_NAME").distinct().count()

125

SELECT COUNT(DISTINCT(ORIGIN_COUNTRY_NAME, DEST_COUNTRY_NAME)) FROM dfTable

<h4>随机抽样</h4>

In [50]:
seed = 5
withReplacement = False
fraction = 0.5
df.sample(withReplacement, fraction, seed).count()

138

<h4>随即分割</h4>

- 将原始DataFrame随即分割为多个切片
- 在机器学习算法中用于分割数据集来创建训练集、验证集和测试集
- 如果一个DataFrame的分割比例和不为1，则比例参数会被自动归一化

In [54]:
dataFrames = df.randomSplit([0.25,0.75], seed=5)
dataFrames[0].count() > dataFrames[1].count()

False

<h4>连接和追加行（联合操作）</h4>

- DataFrame是不可变的，向DataFrame追加行，必须将原DataFrame和新的DataFrame联合起来
- 我们需要使用union操作，并确保两个df有相同的模式和列数

In [66]:
from pyspark.sql import Row
schema = df.schema
newRows = [
    Row("New Country", "Other Country", 5),
    Row("New Country", "Other Country", 1)
]
parallelizedRows = spark.sparkContext.parallelize(newRows)
newDF = spark.createDataFrame(parallelizedRows, schema)

In [67]:
newDF.show()

Py4JJavaError: An error occurred while calling o283.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 61.0 failed 1 times, most recent failure: Lost task 0.0 in stage 61.0 (TID 858) (26.26.26.1 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:182)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:145)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:373)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:337)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoin

In [68]:
df.union(newDF).where("count = 1").where(col("ORIGIN_COUNTRY_NAME") != "United States").show()

AttributeError: 'RDD' object has no attribute '_jdf'

<h4>行排序</h4>

- sort和orderBy方法等价，都是升序排序

In [70]:
df.sort("count").show(5)

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Malta|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|            Croatia|    1|
|       United States|          Gibraltar|    1|
|       United States|          Singapore|    1|
+--------------------+-------------------+-----+
only showing top 5 rows



In [71]:
df.orderBy("count").show(5)

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Malta|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|            Croatia|    1|
|       United States|          Gibraltar|    1|
|       United States|          Singapore|    1|
+--------------------+-------------------+-----+
only showing top 5 rows



In [73]:
from pyspark.sql.functions import desc, asc

df.orderBy(expr("count desc")).show(2)
df.orderBy(col("count").desc(), col("DEST_COUNTRY_NAME").asc()).show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|          Moldova|      United States|    1|
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+
only showing top 2 rows

+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
|    United States|      United States|370002|
|    United States|             Canada|  8483|
+-----------------+-------------------+------+
only showing top 2 rows



-- in SQL<br>
SELECT * FROM dfTable ORDER BY count DESC, DEST_COUNTRY_NAME ASC LIMIT 2

- 最好在进行转换操作前，先对每个分区进行内部排序，可以使用sortWithinPartitions实现

In [4]:
df = spark.read.format("json").load("./data/flight-data/json/2015-summary.json").sortWithinPartitions("count")

In [ ]:
df